In [1]:
import numpy as np
import pickle
import matplotlib

In [2]:
def softmax(m):
    x,y=np.shape(m)
    #生成一个10*10 的随机二维数组，再加上1000  
    #axis=1 表示在二维数组中沿着横轴进行取最大值的操作  
    #m_row_max = m.max(axis = 1)   
    #每一行减去自己本行最大的数字，用到broadcast，reshape  
    #不这么处理会导致INF  
    #m = m - m_row_max.reshape(x,1)

    #计算e的指数次幂  
    m_exp = np.exp(m)  

    #对每一行进行一次求和操作  
    m_exp_row_sum = m_exp.sum(axis=1).reshape(x,1)
    #每一行的原始数据m_exp / 每一行的和  
    softmax_ = m_exp / m_exp_row_sum  
    return (softmax_)  

In [3]:
m = np.array(([1.0,2.0,3.0],[3.0,2.0,1.0]))
print (np.sum(softmax(m),axis =1))

[ 1.  1.]


In [4]:
def one_hot(value):
    n_values = np.max(value)+1
    return np.eye(n_values)[value]

In [5]:
m = np.array([1,2,3])
print (one_hot(m))

[[ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]]


In [6]:
print (0*float('inf'))

nan


In [7]:
# y: real a: pred
def get_loss(y,a):
    e = -np.sum(y.dot(np.log(a.T)))
    return e

In [ ]:
matplotlib.use('Agg')
import matplotlib.pyplot as plt

# Definition of functions and parameters

# NEPOCHis batch size; D_in is input dimension;
# H1 is hidden layer1 dimension; 
# H2 is hidden layer2 dimension
# D_out is output dimension.
D_in = 784
D_out =10
EPOCH = 100
H1 = 300
H2 = 100

learning_rate = 0.1
lam = 0.0005

# Read all data from .pkl
(train_images, train_labels, test_images, test_labels) = pickle.load(open('./mnist_data/data.pkl', 'rb'),
                                                                     encoding='latin1')

### 1. Data preprocessing: normalize all pixels to [0,1) by dividing 256
train_images = train_images/256
test_images = test_images/256

### 2. Weight initialization: Xavier
w1 = (np.random.rand(D_in,H1)-0.5)*(6**0.5)/(H1+D_in)**0.5
w2 = (np.random.rand(H1,H2)-0.5)*(6**0.5)/(H2+H1)**0.5
w3 = (np.random.rand(H2,D_out)-0.5)*(6**0.5)/(H2+D_out)**0.5
# bais
b1 = (np.random.rand(H1,1)-0.5)*(6**0.5)/(H1+1)**0.5
b2 = (np.random.rand(H2,1)-0.5)*(6**0.5)/(H2+1)**0.5
b3 = (np.random.rand(D_out,1)-0.5)*(6**0.5)/(D_out+1)**0.5


### 3. training of neural network
loss = np.zeros((EPOCH))
accuracy = np.zeros((EPOCH))

accuracy= []
n = int(len(train_images)/EPOCH)
for epoch in range(0, n):
    #if epoch > 50:
        #learning_rate = 0.01
    # create input and output
    x = train_images[epoch*EPOCH:epoch*EPOCH+EPOCH]
    y = train_labels[epoch*EPOCH:epoch*EPOCH+EPOCH]
    y = one_hot(y)
    
    # Forward propagation
    h1 = (x.dot(w1).T+b1).T
    # perform relu transform
    h1 = np.maximum(h1,0)
    
    h2 = (h1.dot(w2).T+b2).T
    h2 = np.maximum(h2,0)
    
    h3 = (h2.dot(w3).T+b3).T
    #print('%d:'%epoch+'before'+'*'*100)
    #print(h3)
    h3 = softmax(h3)
    #print('softmax'+'*'*100)
    #print(h3)
    
    #print('softmax_sum'+'*'*100)
    #print(np.sum(h3,axis =1))
    # compute loss function
    
    e = -np.sum(y.dot(np.log(h3.T)))
    print (e/EPOCH)
    l = e/EPOCH + 0.5*lam*(np.sum(np.square(w1)) + np.sum(np.square(w2))+ np.sum(np.square(w3)))
    print(l)
    # Back propagation
    k3 = h3-y
    h2_f = np.where(h2 > 0, 1, 0)
    k2 = k3.dot(w3.T)*h2_f
    h1_f = np.where(h1>0,1,0)
    k1 = k2.dot(w2.T)*h1_f
    
    # Gradient update
    w3 = w3 - learning_rate*h2.T.dot(k3)/EPOCH - learning_rate*lam*w3
    b3 = b3 - (learning_rate*np.sum(k3,axis=0)/EPOCH).reshape(D_out,1)
    w2 = w2 -learning_rate*h1.T.dot(k2)/EPOCH - learning_rate*lam*w2
    b2 = b2 - (learning_rate*np.sum(k2,axis=0)/EPOCH).reshape(H2,1)
    w1 = w1 - learning_rate*x.T.dot(k1)/EPOCH - learning_rate*lam*w1
    b1 = b1 - (learning_rate*np.sum(k1,axis=0)/EPOCH).reshape(H1,1)
    
    # Testing for accuracy
    same_num = 0
    a = np.zeros(np.shape(h3))
    for row in range(np.shape(h3)[0]):
        x = np.argmax(h3[row])
        a[row][x] = 1
        if y[row][x] ==1:
            same_num = same_num +1
    print(same_num/EPOCH)
    
    accuracy.append(same_num/EPOCH)

### 4. Plot
# for example
fig = plt.figure(figsize=(12,5))
fig.suptitle('accuracy') 
ax1 = plt.subplot(111)
#ax1.plot(......)
plt.plot(range(n),accuracy,label='accuracy')
plt.xlabel('time')
plt.ylabel('accuracy')
plt.grid()
plt.legend()
plt.show()
#plt.tight_layout()
plt.savefig('figure.pdf', dbi=300)


235.880889948
235.918609551
0.11
235.763856832
235.801574094
0.08
234.125223814
234.162938898
0.1
235.89484677
235.932561501
0.07
234.855130255
234.892843788
0.08
234.067588049
234.105302362
0.1
232.50796278
232.545676989
0.09
231.748714148
231.786429201
0.11
232.441578407
232.479295666
0.17
234.034515927
234.072237044
0.19
233.908351673
233.946077384
0.09
233.89171854
233.929447643
0.24
232.331439115
232.369173845
0.3
232.959662241
232.99740359
0.26
232.659272925
232.697024066
0.28
230.74207858
230.779838993
0.36
234.237937227
234.275706837
0.31
229.055584421
229.093366996
0.49
231.997305503
232.035100447
0.34
231.523979369
231.561789087
0.35
230.450187864
230.488015759
0.43
232.785705004
232.823549218
0.41
229.655020908
229.69288695
0.53
232.318710165
232.356596144
0.43
230.512344974
230.55025049
0.54
232.20770348
232.245631393
0.49
232.716151215
232.754108301
0.46
234.471782938
234.509767644
0.44
230.636563655
230.67458032
0.64
233.508391074
233.546440225
0.48
231.220383052
231.2584

In [9]:
sum(list([0.04779658,0.0577835 ,0.11268435 ,0.0624714,0.11941648,0.15709186
   ,0.11230297,0.15308466,0.10943788,0.06793031]))

0.9999999900000001

In [10]:
m = np.array(([1.0,2.0,3.0],[3.0,2.0,1.0]))
# 0 : column
for i in range(2):
    m[i]= m[i]/np.sum(m[i])
print (m)

[[ 0.16666667  0.33333333  0.5       ]
 [ 0.5         0.33333333  0.16666667]]


In [11]:
values = [1, 0, 3]
n_values = np.max(values) + 1
np.eye(n_values)[values]

array([[ 0.,  1.,  0.,  0.],
       [ 1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.]])

In [62]:
y = np.array([1,10])
a = np.array(([1.0,2.0,3.0],[3.0,2.0,1.0]))
b = np.array(([1.0,2.0,3.0],[3.0,2.0,1.0]))
print(a*b)
print (a*y.reshape(2,1))
print (np.sum(a,axis =0))

[[ 1.  4.  9.]
 [ 9.  4.  1.]]
[[  1.   2.   3.]
 [ 30.  20.  10.]]
[ 4.  4.  4.]


In [13]:
def loss(y,a):
    return -np.sum(y.dot(np.log(a.T)))

In [14]:
print(loss(y,a))

ValueError: shapes (2,) and (3,2) not aligned: 2 (dim 0) != 3 (dim 0)

In [48]:
# -*- coding: utf-8 -*-
import numpy as np

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random input and output data
x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

# Randomly initialize weights
w1 = np.random.randn(D_in, H)
w2 = np.random.randn(H, D_out)

learning_rate = 1e-6
for t in range(500):
    # Forward pass: compute predicted y
    h = x.dot(w1)
    h_relu = np.maximum(h, 0)
    y_pred = h_relu.dot(w2)

    # Compute and print loss
    loss = np.square(y_pred - y).sum()
    print(t, loss)

    # Backprop to compute gradients of w1 and w2 with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h < 0] = 0
    grad_w1 = x.T.dot(grad_h)

    # Update weights
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

0 30506412.605
1 25706430.4036
2 25397791.8236
3 25363822.8186
4 23255394.3185
5 18511286.4303
6 12741474.0396
7 7814574.22985
8 4568377.68082
9 2697825.92125
10 1692916.39168
11 1149064.18796
12 843249.468185
13 658104.786158
14 536318.999883
15 449611.135241
16 383963.301642
17 331888.40688
18 289358.185618
19 253921.18117
20 223984.960207
21 198457.022805
22 176500.412102
23 157485.343088
24 140946.815001
25 126510.921524
26 113885.411122
27 102769.485296
28 92949.6814643
29 84249.0629755
30 76519.6922045
31 69628.8460272
32 63479.206377
33 57974.2259434
34 53039.914146
35 48606.3186616
36 44612.6970216
37 41005.9876713
38 37744.8197075
39 34788.2867932
40 32102.637708
41 29658.3840603
42 27432.0948063
43 25400.1338184
44 23543.4645926
45 21843.9823903
46 20286.7718686
47 18857.5875403
48 17544.5688456
49 16336.3325027
50 15223.7649789
51 14198.5166504
52 13251.9184142
53 12377.4054155
54 11568.9612936
55 10820.8869233
56 10127.5483747
57 9484.11625597
58 8887.01162937
59 8332.32454

463 0.000174978979797
464 0.000168051778674
465 0.000161396473428
466 0.000155004495391
467 0.000148865609739
468 0.00014297001905
469 0.000137308751531
470 0.000131872916577
471 0.000126652843159
472 0.000121639255714
473 0.000116823103321
474 0.000112198156907
475 0.000107757271015
476 0.000103493013697
477 9.93963061772e-05
478 9.54630277221e-05
479 9.1684806149e-05
480 8.80558120381e-05
481 8.45722022258e-05
482 8.12265447498e-05
483 7.80122571714e-05
484 7.49256116606e-05
485 7.19614408316e-05
486 6.91143272994e-05
487 6.63805660514e-05
488 6.37550812975e-05
489 6.12329283966e-05
490 5.88112890844e-05
491 5.6486015822e-05
492 5.42518055844e-05
493 5.21067893786e-05
494 5.00462417468e-05
495 4.80669222932e-05
496 4.61660899884e-05
497 4.43409103608e-05
498 4.25877456651e-05
499 4.09046668894e-05
